In [11]:
import pandas as pd

# Cargar los archivos CSV proporcionados
df_transferencias = pd.read_csv('0transferencias.csv')
df_saldos = pd.read_csv('0saldos.csv')
df_clientes = pd.read_csv('0clientes.csv')

# Mostrar las primeras filas de cada dataframe para entender su estructura
df_transferencias.head(), df_saldos.head(), df_clientes.head()

C:\Users\Antonio\AppData\Local\Temp\ipykernel_8664\1445608814.py:4: DtypeWarning: Columns (5,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transferencias = pd.read_csv('0transferencias.csv')


(      NroEvento  NroTransaccion FechaEfectiva FechaProceso  ValorNeto Usuario  \
 0  210729985596    210729007960    2021-07-27   2021-07-29 -7921800.0     NaN   
 1  210729986040    210729008404    2021-07-27   2021-07-29 -7850000.0     NaN   
 2  210729985625    210729007989    2021-07-28   2021-07-29 -6000000.0     NaN   
 3  210729985594    210729007958    2021-07-28   2021-07-29 -8000000.0     NaN   
 4  210729986049    210729008413    2021-07-22   2021-07-29 -8256019.0     NaN   
 
    CodDestino  CodTipoGiro CodBancoDestinoACH  CodBancoSkandia TipoCtaACH  \
 0           0            3                NaN              NaN        NaN   
 1           0            3                NaN              NaN        NaN   
 2           0            3                NaN              NaN        NaN   
 3           0            3                NaN              NaN        NaN   
 4           0            3                NaN              NaN        NaN   
 
                TipoOper EstadoRever

In [12]:
# Fusionar los datos de transferencias y saldos basándose en 'Contrato' y 'PlanProducto' para vincular transacciones con los saldos de las cuentas
df_merged = pd.merge(df_transferencias, df_saldos, on=['Contrato', 'PlanProducto'], how='left')

# Calcular las retiradas netas para cada cliente a lo largo del tiempo, considerando tanto aportaciones como retiradas
# Consideraremos el 'ValorNeto' de los datos de transferencias para representar las transacciones
# Luego agregaremos estos para obtener un total por cliente durante los periodos relevantes

# Agregar las transferencias netas por cliente (NroDocum) para obtener las retiradas o aportaciones netas
df_net_transfers = df_merged.groupby('NroDocum')['ValorNeto'].sum().reset_index()

# Fusionar las transferencias netas agregadas con la información del cliente para obtener un conjunto de datos completo por cliente
df_final = pd.merge(df_clientes, df_net_transfers, on='NroDocum', how='left')

# Ahora necesitamos calcular si un cliente ha retirado el 70% o más de su saldo
# Necesitamos un saldo de referencia y las transferencias netas totales para calcular esto, pero parece que necesitamos inferir
# el saldo total de los datos de saldos para establecer la línea base para calcular el umbral del 70%

# Dado que no tenemos un saldo total explícito, supongamos que el saldo más reciente podría servir como referencia por ahora
# Normalmente, necesitaríamos entender claramente cuál saldo usar (por ejemplo, al final de un período específico)

# Simularemos el cálculo del saldo solo para tener una lógica lista, normalmente esto sería más complejo y específico de los datos
# Para simplificar, asumiré el saldo conocido más reciente para cada cliente como su saldo de referencia para el cálculo del 70%

# Supongamos que 'SALDO_202202' es el saldo más reciente disponible para simplificación en este contexto
# Fusionando la información del saldo más reciente con las transferencias netas
df_balance = df_saldos[['NroDocum', 'SALDO_202210']].rename(columns={'SALDO_202210': 'LatestBalance'})  # Asumiendo 'SALDO_202210' como el saldo más reciente
df_final = pd.merge(df_final, df_balance, on='NroDocum', how='left')

# Calcular la bandera para retirada del 70% o más
df_final['Withdrawal_70_Flag'] = (df_final['ValorNeto'].abs() >= df_final['LatestBalance'] * 0.70).astype(int)

df_final.head()

,TIPODOCUM,CIUDAD,FecNacim,EnvioExtractos,NroDocum,ValorNeto,LatestBalance,Withdrawal_70_Flag
0,C,"BOGOTA D.C., BOGOTA",1954-12-16,E,1092070504,-19091200.00,13382658.87,1
1,C,"BOGOTA D.C., BOGOTA",1969-08-04,E,1097627287,-13000000.00,0.00,1
2,C,"BOGOTA D.C., BOGOTA",1969-08-04,E,1097627287,-13000000.00,76451290.69,0
3,C,"BOGOTA D.C., BOGOTA",1961-08-15,T,1066619616,NaN,0.00,0
4,C,"BOGOTA D.C., BOGOTA",1986-02-08,T,1079454835,2734360.41,0.00,1


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Preparar la matriz de características (X) y la variable objetivo (y)
X = df_final[['ValorNeto', 'LatestBalance']]  # Conjunto de características simplificado para la demostración
y = df_final['Withdrawal_70_Flag']

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# Rellenar los valores NaN con 0 para 'ValorNeto' y 'LatestBalance' asumiendo que NaN significa que no hay movimiento o saldo
X_filled = X.fillna(0)

# Dividir el conjunto de datos en conjuntos de entrenamiento y prueba de nuevo después de rellenar los valores NaN
X_train_filled, X_test_filled, y_train, y_test = train_test_split(X_filled, y, test_size=0.3, random_state=42)

# Normalizar las características después de rellenar los valores NaN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filled)
X_test_scaled = scaler.transform(X_test_filled)

# Ajustar los modelos con los datos actualizados
logistic_model.fit(X_train_scaled, y_train)
random_forest_model.fit(X_train_filled, y_train)  # No es necesario escalar características para modelos basados en árboles
gbm_model.fit(X_train_filled, y_train)

# Hacer predicciones sobre el conjunto de prueba
y_pred_logistic = logistic_model.predict(X_test_scaled)
y_pred_rf = random_forest_model.predict(X_test_filled)
y_pred_gbm = gbm_model.predict(X_test_filled)

# Calcular las métricas de evaluación para cada modelo
metrics = {
    'Modelo': ['Regresión Logística', 'Random Forest', 'Gradient Boosting'],
    'Exactitud': [accuracy_score(y_test, y_pred_logistic),
                  accuracy_score(y_test, y_pred_rf),
                  accuracy_score(y_test, y_pred_gbm)],
    'Precisión': [precision_score(y_test, y_pred_logistic),
                  precision_score(y_test, y_pred_rf),
                  precision_score(y_test, y_pred_gbm)],
    'Recuperación': [recall_score(y_test, y_pred_logistic),
                     recall_score(y_test, y_pred_rf),
                     recall_score(y_test, y_pred_gbm)],
    'Puntaje F1': [f1_score(y_test, y_pred_logistic),
                   f1_score(y_test, y_pred_rf),
                   f1_score(y_test, y_pred_gbm)]
}

pd.DataFrame(metrics)

,Modelo,Exactitud,Precisión,Recuperación,Puntaje F1
0,Regresión Logística,0.814746,0.982036,0.014430,0.028443
1,Random Forest,0.998776,0.997796,0.995689,0.996741
2,Gradient Boosting,0.996660,0.987254,0.995073,0.991148
